# README? NoADME!

## Overview

A concept figure?

In [ ]:
!pip3 install -qU guardrails-ai openai langchain_community langchain_experimental langchain-upstage sentence-transformers langchainhub langchain-chroma langchain matplotlib python-dotenv tavily-python ragas faiss-cpu tokenizers gradio

In [ ]:
%load_ext dotenv
%dotenv
# set UPSTAGE_API_KEY

In [ ]:
import warnings

warnings.filterwarnings("ignore")

In [ ]:
import gradio as gr

from langchain_upstage import ChatUpstage
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.output_parsers import StrOutputParser
from langchain.schema import AIMessage, HumanMessage


llm = ChatUpstage(streaming=True)

In [ ]:
# More general chat
chat_with_history_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "You are a helpful assistant."),
        MessagesPlaceholder(variable_name="history"),
        ("human", "{message}"),
    ]
)

# Need prompt code


In [ ]:
# from langchain_core.prompts import PromptTemplate
# from langchain_upstage import ChatUpstage
# from langchain_core.output_parsers import StrOutputParser

# prompt_template = PromptTemplate.from_template(
#     """
#     Q: Please extract user and participants's name  from the following text. I am a user. Then, organize the action relationships between me and the participants. 

#     ---
#     We are a team of 5 members called Krispy Team.
#     Dr. Kim Yoon-gon gave me coffee. I said thank you.
#     Student Jae-min also received coffee from Dr. Kim Yoon-gon. Dr. Son Seok-ho is explaining the team project topic now.
#     Student Sung-hoon is preparing presentation materials for other team members.

#     A:
#     sequenceDiagram
#         actor User
#         participant Kim Yoon-gon
#         participant Jae-min
#         participant Son Seok-ho
#         participant Sung-hoon
#         participant Other team members

#         User ->> Kim Yoon-gon : Received coffee and said thank you
#         Kim Yoon-gon ->> Jae-min : Gave coffee
#         Son Seok-ho ->> Team : Explained the project topic
#         Sung-hoon ->> Other team members : Prepared presentation materials

#     Q: Please extract user and participants's name  from the following text. I am a user.Then, organize the action relationships between me and the participants. 

#     ---
#     {text}
#     """
# )

# llm = ChatUpstage()
# chain = prompt_template | llm | StrOutputParser()
# content = "We are a team of four.I am currently creating a template to give to Dr. Kim Yoon-gon.Also, Student Sung-hoon is creating an RAG to give to Dr. Kim Yoon-gon. The other two members are discussing ideas with each other."
# answer = chain.invoke({"text": content})
# print(answer)

# Need knowledge base and RAG

# Tool RAG


In [ ]:
chain = chat_with_history_prompt | llm | StrOutputParser()

In [ ]:
def chat(message, history):
    history_langchain_format = []
    for human, ai in history:
        history_langchain_format.append(HumanMessage(content=human))
        history_langchain_format.append(AIMessage(content=ai))

    generator = chain.stream({"message": message, "history": history_langchain_format})

    assistant = ""
    for gen in generator:
        assistant += gen
        yield assistant

In [ ]:
with gr.Blocks() as demo:
    chatbot = gr.ChatInterface(
        chat,
        examples=[
            "How to eat healthy?",
            "Best Places in Korea",
            "How to make a chatbot?",
        ],
        title="Solar Chatbot",
        description="Upstage Solar Chatbot",
    )
    chatbot.chatbot.height = 300

In [ ]:
if __name__ == "__main__":
    demo.launch()